# Time propagation of a wavepacket: <br/> convergence of the error estimation of the Siegert states expansions

This notebooks shows more refined calculations than the `time_propagation_error_estimation.ipynb` notebook. Each parameter that might affect the error estimation is varied separately, to have a better understanding on the underlying convergence as a function of all these parameters and get an idea of which values should be used to get moderately accurate results. Note that it only concentrates on the error of the exact Sigert states expansion, though.

**Import useful modules and classes.**

In [ ]:
# Make the notebook aware of some of the SiegPy module classes
from siegpy import SWPBasisSet, Gaussian
# Other imports
import numpy as np

**Define useful functions**

In [ ]:
def print_errors(exact, siegert_exp, dx):
    """
    Function printing the absolute and relative error of a given 
    Siegert states expansion of the time propagation of a wave-packet
    for different times.
    
    :param exact: Exact time-propagation of a wavepacket.
    :type exact: 2D numpy.array
    :param siegert_exp: Siegert states expansion of the time-
                        propagation of the same wavepacket.
    :type siegert_exp: 2D numpy.array
    :param dx: Space grid-step.
    :type: float
    """
    # Print lines to initialize the table
    title = "  t    abs. error   rel. error"
    line = "-"*len(title)
    print(line)
    print(title)
    print(line)
    # Get the error estimations for each time
    for i, t in enumerate(time_grid):
        abs_err = np.trapz(np.abs(exact[i] - siegert_exp[i]), dx=dx)
        norm = np.trapz(np.abs(exact[i]), dx=dx)
        #abs_err = np.trapz(np.abs(exact[i] - siegert_exp[i])**2, dx=dx)
        #norm = np.trapz(np.abs(exact[i])**2, dx=dx)
        rel_err = abs_err / norm
        print("{:.2f}   {: .3e}   {: .3e}".format(t, abs_err, rel_err))
    # Print lines to finalize the table
    print(line)
    print()
    
    
def SSE_errors(filename, nres, k_max, h_k, nx, test_func, time_grid, \
             choice={'exact': True, 'MLE': True, 'Ber': True}):
    """
    Function performing the time-propagation of an initial wave-packet
    using different Siegert states expansions and printing the absolute 
    and relative errors when compared to the exact time-propagation of 
    this initial wavepacket. 
    
    :param filename: Name of the file where to look for the Siegert states.
    :type filename: str
    :param nres: Number of resonant states to use in the Siegert states expansions.
    :type nres: int
    :param k_max: Wavenumber of the last continuum state involved in the exact calculations.
    :type k_max: float
    :param h_k: Wavenumber grid step of the continuum basis set.
    :type h_k: float
    :param nx: Number of space grid points.
    :type nx: int
    :param test_func: Initial wave-packet.
    :type test_func: Wavefunction
    :param time_grid: Times at which the
    :type time_grid: list or numpy.array
    :param choice: Optional, used to say which Siegert state expansion is desired.
    :type choice: dict
    """
    # Siegert states basis set initialized from a file
    siegerts = SWPBasisSet.from_file(filename, nres=nres)
    
    # Read the potential from a data file
    potential = siegerts[0].potential
    l = potential.width
    
    # Define a grid space
    xgrid = np.linspace(-l/2, l/2, nx)
    dx = xgrid[1] - xgrid[0]
    
    # Discretize the Siegert states over the grid
    for s in siegerts:
        s.grid = xgrid

    # Exact basis set
    cont = SWPBasisSet.find_continuum_states(potential, k_max, h_k, grid=xgrid)
    exact = cont + siegerts.bounds

    # Evaluation of the exact time propagation
    exact_tp = exact.exact_propagation(test_func, time_grid)
    
    if choice['exact']:
        # Evaluation of the exact Siegert expansion of the time propagation
        exact_S_tp = siegerts.exact_Siegert_propagation(test_func, time_grid)
        # Find the error with respect ot the exact time propagation
        print("         Exact Siegert")
        print_errors(exact_tp, exact_S_tp, dx)

    if choice['MLE']:
        # Evaluation of the Mittag-Leffler expansion of the time propagation
        MLE_tp = siegerts.MLE_propagation(test_func, time_grid)
        # Find the error with respect ot the exact time propagation
        print("              MLE")
        print_errors(exact_tp, MLE_tp, dx)

    if choice['Ber']:
        # Evaluation of the Berggren expansion of the time propagation
        Ber_tp = siegerts.Berggren_propagation(test_func, time_grid)       
        # Find the error with respect ot the exact time propagation 
        print("            Berggren")
        print_errors(exact_tp, Ber_tp, dx)

## Define a potential

The potential under consideration and its Siegert states are stored in a file.

In [ ]:
# Siegert states basis set initialized from a file
filename = 'siegerts.dat'
siegerts = SWPBasisSet.from_file(filename)

# Define the potential
potential = siegerts[0].potential
l = potential.width

## Define a time grid

In [ ]:
time_grid = np.linspace(0, 3, 13)

## Case 1: initial wave-packet with a small initial momentum

The first initial wave-packet considered uses a small initial momentum.

In [ ]:
l = potential.width
sigma = l/20. # width of the Gaussian
x_c   = -l/4. # center of the Gaussian
k_0   = 1.    # initial momentum
gauss_small_momentum = Gaussian(sigma, x_c, k0=k_0)

### Influence of the space grid

In [ ]:
for nx in [201, 401, 801, 1601]:
    print(nx)
    SSE_errors(filename, 50, 40, 0.01, nx, gauss_small_momentum, time_grid, \
             choice={'exact': True, 'MLE': False, 'Ber': False})
    #SSE_errors(filename, 50, 40, 0.01, nx, gauss_small_momentum, time_grid)

A large number of space grid points is necessary for the error estimation to be considered as converged with respect to this parameter. Still, using a rather small grid with 201 points already gives a correct error estimation.

### Influence of the range of the continuum grid

In [ ]:
for kmax in [40, 80]:
    print(kmax)
    SSE_errors(filename, 50, kmax, 0.01, 1601, gauss_small_momentum, time_grid, \
             choice={'exact': True, 'MLE': False, 'Ber': False})
    #SSE_errors(filename, 50, kmax, 0.01, 1601, gauss_small_momentum, time_grid)

The exact time-propagation could already be considered as converged with respect to the range of the wavenumbers of continuum states.

### Influence of the wavenumber step of the continuum grid

In [ ]:
for hk in [0.1, 0.05, 0.01, 0.005]:
    print(hk)
    SSE_errors(filename, 50, 40, hk, 1601, gauss_small_momentum, time_grid, \
             choice={'exact': True, 'MLE': False, 'Ber': False})
    #SSE_errors(filename, 50, 40, hk, 1601, gauss_small_momentum, time_grid)

If the wavenumber grid is not dense enough, then the error estimation can be off by more than an order of magnitude. To reach a convergence with respect to this parameter is difficult (just like the convergence with respect to the space grid density).

### Influence of the number of Siegert states

We increase the range of the wavenumbers of the continuum states and test the influence of the number of Siegert states.

In [ ]:
for n_res in [25, 50, 78]:
    print(n_res)
    SSE_errors(filename, n_res, 60, 0.005, 1601, gauss_small_momentum, time_grid, \
             choice={'exact': True, 'MLE': False, 'Ber': False})
    #SSE_errors(filename, n_res, 200, 0.01, 1601, gauss_small_momentum, time_grid)

There is no difference in using 50 or 78 Siegert states, the error estimation is converged with respect to this parameter. Also note that using 50 or 78 Siegert states with a maximal continuum states wavenumber $k_{max} = 60$ gives the same error as in the previous calculation using 50 Siegert states and $k_{max} = 40$.

## Case 2: initial wave-packet with large initial momentum

Similar calculations can be produced in this case. The trends being similar than in the previous case, it is more interesting to see how both cases compare.

In [ ]:
k_0 = 20. # initial momentum
gauss_large_momentum = Gaussian(sigma, x_c, k0=k_0)

### Influence of the space grid

In [ ]:
for nx in [201, 401, 801, 1601]:
    print(nx)
    SSE_errors(filename, 50, 40, 0.01, nx, gauss_large_momentum, time_grid, \
             choice={'exact': True, 'MLE': False, 'Ber': False})
    #SSE_errors(filename, 50, 40, 0.01, nx, gauss_large_momentum, time_grid)

It is easier to reach a convergence of the error with respect to the space grid density with this large initial momentum: there is almost no difference between using 201 or 1601 points.

### Influence of the range of the continuum grid

In [ ]:
for kmax in [40, 60, 80]:
    print(kmax)
    SSE_errors(filename, 50, kmax, 0.01, 1601, gauss_large_momentum, time_grid, \
             choice={'exact': True, 'MLE': False, 'Ber': False})
    #SSE_errors(filename, 50, kmax, 0.01, 1601, gauss_large_momentum, time_grid)

The grid of continuum states extending up to wavenumbers $k_{max} = 40$ is already almost converged, but it must be larger than in the case of the smaller initial momentum. This is easily explained by a comparative study of the convergence of the completeness relation of the initial wave-packets with small and large initial momenta.

### Influence of the wavenumber step of the continuum grid

In [ ]:
for hk in [0.1, 0.05, 0.01, 0.005, 0.0025]:
    print(hk)
    SSE_errors(filename, 50, 60, hk, 1601, gauss_large_momentum, time_grid, \
             choice={'exact': True, 'MLE': False, 'Ber': False})
    #SSE_errors(filename, 50, 40, hk, 1601, gauss_large_momentum, time_grid)

A smaller wavenumber grid step is required to reach a convergence. You can also see that the smaller the wavenumber grid step is, the larger the times at which the error estimation is converged ($h_k = 0.1$ is enough for $t=0$, $h_k = 0.05$ is enough up to $t=1.0$ and $h_k = 0.01$ shows converged results at least up to $t = 3.0$).

### Influence of the number of Siegert states

In [ ]:
for n_res in [25, 50, 78]:
    print(n_res)
    SSE_errors(filename, n_res, 60, 0.005, 1601, gauss_large_momentum, time_grid, \
             choice={'exact': True, 'MLE': False, 'Ber': False})
    #SSE_errors(filename, n_res, 200, 0.01, 1601, gauss_large_momentum, time_grid)

Finally, and in the same manner as for the $k_{max}$ parameter, it requires more Siegert states to reach a convergence of the error estimation.